In [297]:
from faker import Faker
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
from random import shuffle
import matplotlib.pyplot as plt
from scipy.stats import truncnorm


In [298]:
fake = Faker()

In [299]:
#    mu1, sigma1 = 11, 1
    # mu2, sigma2 = 17, 1

In [300]:
# Weekday
# 11am, 5pm
def weekday(mu1, sigma1, mu2, sigma2, N):
    X1 = np.random.normal(mu1, sigma1, int(N/2))
    X2 = np.random.normal(mu2, sigma2, int(N/2))
    X = np.concatenate([X1,X2])
    return X


# px.histogram(X, nbins=40)

In [301]:
# Limit normal distribution to operating times (when clinic is open)
def get_truncated_normal(mean=13, sd=3, low=8, upp=20):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [302]:
#  mu, sigma = 13, 3

In [303]:
def weekend(mu, sigma, N):
    X = get_truncated_normal()
    X = X.rvs(N)
    return X


In [304]:
#    mu1, sigma1 = 11, 1
    # mu2, sigma2 = 17, 1

In [305]:
# Denver
X_denver = []
for i in range(1,101):
    # Weekend
    if (i % 7 == 6) or (i % 7 == 0):    
        X_denver.extend(weekend(13, 3, 60))
    # Weekday
    else:                               
        X_denver.extend(weekday(11, 1, 17, 1, 60))

print(len(X_denver))

6000


In [306]:
# Lakewood
X_lakewood = []
for i in range(1,101):
    # Weekend
    if (i % 7 == 6) or (i % 7 == 0):    
        X_lakewood.extend(weekend(13, 3, 50))
    # Weekday
    else:                               
        X_lakewood.extend(weekday(11, 1, 17, 1, 50))

print(len(X_lakewood))

5000


In [307]:
# Wheatridge
X_wheatridge = []
for i in range(1,101):
    # Weekend
    if (i % 7 == 6) or (i % 7 == 0):    
        X_wheatridge.extend(weekend(13, 3, 30))
    # Weekday
    else:                               
        X_wheatridge.extend(weekday(11, 1, 17, 1, 30))

print(len(X_wheatridge))

3000


In [308]:
# Edgewater
X_edgewater = []
for i in range(1,101):
    # Weekend
    if (i % 7 == 6) or (i % 7 == 0):    
        X_edgewater.extend(weekend(13, 3, 30))
    # Weekday
    else:                               
        X_edgewater.extend(weekday(11, 1, 17, 1, 30))

print(len(X_edgewater))

3000


In [309]:
# RiNO
X_rino = []
for i in range(1,101):
    # Weekend
    if (i % 7 == 6) or (i % 7 == 0):    
        X_rino.extend(weekend(13, 3, 30))
    # Weekday
    else:                               
        X_rino.extend(weekday(11, 1, 17, 1, 30))

print(len(X_rino))

3000


In [310]:
X_all = X_denver + X_edgewater + X_wheatridge + X_rino + X_lakewood

In [311]:
# weekday mornings before 10am
# weekday lunch 12-2pm
# weekday evening 4pm-8pm

# weekends 11am-8pm, peak at 3pm

# For 20k patients, we have 100 days (~3months of data) - 1st 100 days of 2022
# 28 days are weekends (sat/sun)
# 72 days are weekdays

In [312]:
# l
loc = {
    'denver': 6000,
    'edgewater': 3000,
    'wheatridge': 3000,
    'rino':3000,
    'lakewood': 5000}

l = []
for k, v in loc.items():
    for i in range(v):
        l.append(k)

l  

['denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',
 'denver',

In [313]:
# We took the shortest distances google map directions gave us

cities = ['denver', 'edgewater', 'wheatridge', 'rino', 'lakewood']
lat = []
lon = []
# Denver
locations = pd.DataFrame({
    'to_denver': [0, 5, 6.3, 1.9, 7.5],
    'to_edgewater': [5.1, 0, 2, 7.8, 4.3],
    'to_wheatridge': [6.3, 2, 0, 7.5, 12],
    'to_rino':[2, 7.7, 7.8, 0, 11],
    'to_lakewood':[8, 4.4, 4.8, 10.7, 0]},
    index=cities) 

locations['lat'] = [39.73906432357836, 39.753954449845445, 39.76685732722651, 39.767327859566265, 39.70455155721396]
locations['lon'] = [-104.98969659655802, -105.06778796142915, -105.08198265044479, -104.98113186098168, -105.0798829449297]
locations

,to_denver,to_edgewater,to_wheatridge,to_rino,to_lakewood,lat,lon
denver,0.0,5.1,6.3,2.0,8.0,39.739064,-104.989697
edgewater,5.0,0.0,2.0,7.7,4.4,39.753954,-105.067788
wheatridge,6.3,2.0,0.0,7.8,4.8,39.766857,-105.081983
rino,1.9,7.8,7.5,0.0,10.7,39.767328,-104.981132
lakewood,7.5,4.3,12.0,11.0,0.0,39.704552,-105.079883


### Patients
- Patient ID
- Last, First Name
- DOB > Age
- Reason for visit
- Location they visited


In [314]:
pids = []
for pid in range(10000, 30000):
    pids.append(pid)


In [315]:
# Reasons for visit
reasons = {
    'cold/flu':3000,
    'covid':3000,
    'stomach':3000,
    'cough/chest':3000,
    'injury':1000,
    'throat':1000,
    'back_pain':3000,
    'blood':1000,
    'eyes':1000,
    'ears':1000,
}

r = []
for k, v in reasons.items():
    for i in range(v):
        r.append(k)

shuffle(r)
# r     

In [316]:
first_names = []
last_names = []

for i in range(10000, 30000):
    first = "first_"+str(i)
    last = "last_"+str(i)
    first_names.append(first)
    last_names.append(last)
    

In [317]:
# Date of Births

dobs = []

# Ages < 18 - 10% - 2000
for _ in range(2000):
    dobs.append(fake.date_of_birth(minimum_age=0, maximum_age=17))

# Ages 18-30 - 20% - 4000
for _ in range(4000):
    dobs.append(fake.date_of_birth(minimum_age=18, maximum_age=30))

# Ages 31-40 - 20% - 4000
for _ in range(4000):
    dobs.append(fake.date_of_birth(minimum_age=31, maximum_age=40))

# Ages 41-50 - 15% - 3000
for _ in range(3000):
    dobs.append(fake.date_of_birth(minimum_age=41, maximum_age=50))

# Ages 51-60 - 15% - 3000
for _ in range(3000):
    dobs.append(fake.date_of_birth(minimum_age=51, maximum_age=60))

# Ages >60 - 20% - 4000
for _ in range(4000):
    dobs.append(fake.date_of_birth(minimum_age=61, maximum_age=85))


In [318]:
dates_all = []
for N in [60, 30, 30, 30, 50]:
    dates = pd.date_range(datetime.date(2022,1,1), periods=100).tolist()
    dates = [i.date().strftime('%x') for i in dates]
    dates = dates*N
    dates.sort()
    dates_all.extend(dates)

In [319]:
len(dates_all)

20000

In [324]:
patients = pd.DataFrame()

patients['pid'] = pids
patients.set_index('pid', inplace=True)
patients['first_name'] = first_names
patients['last_names'] = last_names
patients['birth_date'] = dobs
patients['reason'] = r
patients['location'] = l
patients['visit_date'] = dates_all
patients['checkin_time'] = X_all

patients

,first_name,last_names,birth_date,reason,location,visit_date,checkin_time
pid,,,,,,,
10000,first_10000,last_10000,2020-09-08,back_pain,denver,01/01/22,10.229075
10001,first_10001,last_10001,2009-09-20,covid,denver,01/01/22,11.176226
10002,first_10002,last_10002,2007-02-24,back_pain,denver,01/01/22,11.133643
10003,first_10003,last_10003,2020-01-18,cold/flu,denver,01/01/22,13.092859
10004,first_10004,last_10004,2020-09-06,blood,denver,01/01/22,9.553806
...,...,...,...,...,...,...,...
29995,first_29995,last_29995,1950-07-04,cough/chest,lakewood,04/10/22,18.416605
29996,first_29996,last_29996,1957-03-30,eyes,lakewood,04/10/22,17.272793
29997,first_29997,last_29997,1942-04-17,stomach,lakewood,04/10/22,17.076900


In [345]:

def make_time(x):
    dec = x - int(x)
    hour = int(x)
    min = int(dec*60)
    sec = int(dec*60*60 % 60)
    return datetime.time(hour, min, sec).strftime('%X')

patients['checkin_time'] = patients['checkin_time'].map(make_time)
patients

,first_name,last_names,birth_date,reason,location,visit_date,checkin_time
pid,,,,,,,
10000,first_10000,last_10000,2020-09-08,back_pain,denver,01/01/22,10:13:44
10001,first_10001,last_10001,2009-09-20,covid,denver,01/01/22,11:10:34
10002,first_10002,last_10002,2007-02-24,back_pain,denver,01/01/22,11:08:01
10003,first_10003,last_10003,2020-01-18,cold/flu,denver,01/01/22,13:05:34
10004,first_10004,last_10004,2020-09-06,blood,denver,01/01/22,09:33:13
...,...,...,...,...,...,...,...
29995,first_29995,last_29995,1950-07-04,cough/chest,lakewood,04/10/22,18:24:59
29996,first_29996,last_29996,1957-03-30,eyes,lakewood,04/10/22,17:16:22
29997,first_29997,last_29997,1942-04-17,stomach,lakewood,04/10/22,17:04:36


In [321]:
fig = px.histogram(patients.birth_date)
fig.show()

In [322]:
fig = px.histogram(patients.reason)
fig.show()

In [323]:
fig = px.pie(patients, names='reason')
fig.show()